In [1]:
import numpy as np
import pandas as pd

In [20]:
data = pd.read_csv(r"cancer.csv")
data["type"] = data["type"].map({0:-1,1:1})
data.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension,type
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,-1
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,-1
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,-1
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,-1
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,-1


In [21]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 569 entries, 0 to 568
Data columns (total 31 columns):
mean radius                569 non-null float64
mean texture               569 non-null float64
mean perimeter             569 non-null float64
mean area                  569 non-null float64
mean smoothness            569 non-null float64
mean compactness           569 non-null float64
mean concavity             569 non-null float64
mean concave points        569 non-null float64
mean symmetry              569 non-null float64
mean fractal dimension     569 non-null float64
radius error               569 non-null float64
texture error              569 non-null float64
perimeter error            569 non-null float64
area error                 569 non-null float64
smoothness error           569 non-null float64
compactness error          569 non-null float64
concavity error            569 non-null float64
concave points error       569 non-null float64
symmetry error             569 

In [49]:
class Perceptron:
    '''使用Python语言实现感知器算法'''
    
    def __init__(self, alpha, times):
        self.alpha = alpha
        self.times = times
        
    def step(self, z):
        '''阶跃函数
        
        Parameters
        ----
        z:数组或者标量
            阶跃函数的参数，可以将z映射成1或者-1
        
        Returns
        ----
        value: int
            如果z>=0,返回1，否则返回-1
        '''
        
        return np.where(z >=0, 1, -1)
    
    def fit(self, X, y):
        
        X = np.asarray(X)
        y = np.asarray(y)
        
        self.w_ = np.zeros(1 + X.shape[1])
        
        self.loss_ = []
        
        for i in range(self.times):
            # 逻辑回归中，使用所有样本计算梯度，然后更新权重
            # 而感知器中，是使用单个样本，依次进行计算梯度，更新权重
            loss = 0
            for x, target in zip(X, y):
                # 计算预测值
                y_hat = self.step(np.dot(x, self.w_[1:]) + self.w_[0])
                loss += y_hat != target
                #更新权重:w(j) = w(j) + 学习率 * (真实值 - 预测值) * x(j)
                # 例如，结果应该是1，但是返回了-1，这证明x重要的特征对应w的权重较低，所以要增加w中x重要的特征，所以用(target - y_hat) * x来更新权重
                # 反之如果应该是-1，但是返回了1，就需要降低x相应重要特征的权重，这个时候加一个负数，也就达到了降低的效果
                display(self.alpha)
                self.w_[0] += self.alpha * (target - y_hat)
                self.w_[1:] += self.alpha * (target - y_hat) * x
                
            self.loss_.append(loss)
            
    def predict(self, X):
        '''根据参数传递样本'''
        
        return self.step(np.dot(X, self.w_[1:]) + self.w_[0])

In [50]:
class StandardScaler:
    '''标准化'''
    
    def fit(self, X):
        '''根据传递的样本，计算每个特征列的均值与标准差
        
        Parameters
        ----
        X:类数组
        '''
        
        X = np.asarray(X)
        self.std_ = np.std(X, axis=0)
        self.mean_ = np.mean(X, axis=0)
        
    def transform(self, X):
        '''将每一列都标准化处理，每一列都变成标准正态分布'''
        
        return (X-self.mean_) / self.std_
    
    def fit_transform(self, X):
        '''结合'''
        
        self.fit(X)
        return  self.transform(X)

In [51]:
t1 = data[data["type"] == 1]
t2 = data[data["type"] == -1]
#display(t1.count())
#display(t2.count())
t1 = t1.sample(len(t1), random_state=0)
t2 = t2.sample(len(t2), random_state=0)

train_X = pd.concat([t1.iloc[:250, :-1], t2.iloc[:150, :-1]], axis=0)
train_y = pd.concat([t1.iloc[:250, -1], t2.iloc[:150, -1]], axis=0)
test_X = pd.concat([t1.iloc[250:, :-1], t2.iloc[150:, :-1]], axis=0)
test_y = pd.concat([t1.iloc[250:, -1], t2.iloc[150:, -1]], axis=0)

s = StandardScaler()
train_X = s.fit_transform(train_X)
test_X = s.transform(test_X)

#display(train_X)

p = Perceptron(0.0000000001, 50)
p.fit(train_X, train_y)
result = p.predict(test_X)

display(result)
display(p.loss_)

1e-10

1e-10

1e-10

1e-10

1e-10

...

1e-10

1e-10

1e-10

array([ 1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,
        1,  1,  1,  1,  1,  1,  1,  1,  1, -1,  1,  1,  1,  1,  1,  1, -1,
        1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
        1,  1,  1,  1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1,
       -1, -1, -1, -1, -1, -1,  1, -1, -1, -1, -1, -1, -1, -1, -1, -1])

[7,
 18,
 11,
 10,
 9,
 13,
 8,
 10,
 7,
 12,
 9,
 14,
 12,
 9,
 10,
 10,
 12,
 10,
 13,
 11,
 9,
 8,
 10,
 7,
 11,
 6,
 4,
 12,
 10,
 12,
 8,
 7,
 13,
 9,
 13,
 9,
 10,
 10,
 8,
 9,
 10,
 8,
 10,
 9,
 13,
 10,
 8,
 8,
 7,
 10]